# Soluciones Proyecto ETL

In [1]:
import pandas as pd
import requests
import datetime

## Ejercicio 1

In [2]:
postal_codes_raw = pd.read_csv('./Pill ETL - S01 - postal codes.csv', sep=';', encoding = "utf-8")

In [3]:
data = []
ids = 1
for i in postal_codes_raw.index:
    row = postal_codes_raw.iloc[i]
    
    result = {"id": ids + 1000, "cp": row['Código Postal']}
    ids += 1
    
    prov_names = row['provincia'].replace('?', 'ñ').split('/')
    city_names = row['poblacion'].replace('?', 'ñ').split('/')
    
    if len(prov_names) == 1:
        result['provincia'] = prov_names[0]
        result['provincia_local'] = prov_names[0]
    else:
        result['provincia'] = prov_names[1]
        result['provincia_local'] = prov_names[0]
        
    if len(city_names) == 1:
        result['ciudad'] = city_names[0]
        result['ciudad_local'] = city_names[0]
    else:
        result['ciudad'] = city_names[1]
        result['ciudad_local'] = city_names[0]
        
    data.append(result)
    
cp_table = pd.DataFrame(data, columns= data[0].keys())
    

In [11]:
cp_table.head(5)

,id,cp,provincia,provincia_local,ciudad,ciudad_local
0,1001,240,Álava,Araba,Alegría-Dulantzi,Alegría-Dulantzi
1,1002,548,Ávila,Ávila,Candeleda,Candeleda
2,1003,1001,Álava,Araba,Vitoria-Gasteiz,Vitoria-Gasteiz
3,1004,1002,Álava,Araba,Vitoria-Gasteiz,Vitoria-Gasteiz
4,1005,1003,Álava,Araba,Vitoria-Gasteiz,Vitoria-Gasteiz


## Ejercicio 2

In [5]:
basic_headers = {'User-Agent': 'Mozilla/5.0'}
endpoint = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM'
payload = requests.get(endpoint, headers=basic_headers)

In [6]:
population_data = payload.json()

In [7]:
population_clean = []
pop_err = []
for reg in population_data:
    
    try:
        if reg['MetaData'][1]['Variable']['Codigo'] == 'MUN':
            temp = {"id": reg['COD']}

            temp['Poblacion'] = reg['MetaData'][1]['Nombre']
            temp['Origen'] = reg['MetaData'][2]['Nombre']

            for row in reg['Data']:
                temp[datetime.datetime.strptime(row['Fecha'], '%Y-%m-%dT%H:%M:%S.000+01:00').year] = int(row['Valor'])

            population_clean.append(temp)
    except Exception as e:
        
            pop_err.append(reg)
        

In [12]:
population_std = pd.DataFrame(population_clean, columns=population_clean[0].keys())
population_std.head(20)

,id,Poblacion,Origen,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5662094,Agullana,Total,885.0,863.0,831.0,841.0,831.0,841.0,828.0,...,858.0,839.0,840.0,812.0,806.0,753.0,756.0,759.0,746.0,723.0
1,PC5662093,Agullana,España,711.0,704.0,689.0,697.0,692.0,704.0,686.0,...,678.0,657.0,651.0,645.0,648.0,645.0,662.0,670.0,657.0,664.0
2,PC5662092,Agullana,Extranjero,174.0,159.0,142.0,144.0,139.0,137.0,142.0,...,180.0,182.0,189.0,167.0,158.0,108.0,94.0,89.0,89.0,59.0
3,PC5662063,Agullana,Europa (sin España),79.0,67.0,56.0,57.0,53.0,55.0,64.0,...,94.0,103.0,103.0,92.0,84.0,59.0,51.0,55.0,53.0,33.0
4,PC5662062,Agullana,UE28 sin España,NaN,57.0,50.0,54.0,49.0,51.0,59.0,...,89.0,97.0,96.0,87.0,79.0,56.0,48.0,49.0,46.0,28.0
5,PC5662091,Agullana,Alemania,2.0,1.0,1.0,2.0,1.0,1.0,1.0,...,3.0,4.0,4.0,3.0,3.0,2.0,3.0,3.0,5.0,4.0
6,PC5662090,Agullana,Bulgaria,0.0,2.0,2.0,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,1.0,0.0
7,PC5662089,Agullana,Francia,44.0,35.0,28.0,31.0,28.0,29.0,34.0,...,52.0,51.0,56.0,47.0,45.0,40.0,31.0,32.0,30.0,18.0
8,PC5662088,Agullana,Italia,0.0,1.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,PC5662087,Agullana,Polonia,3.0,0.0,0.0,2.0,2.0,2.0,2.0,...,6.0,11.0,7.0,10.0,9.0,4.0,4.0,1.0,1.0,0.0


## Ejercicio 3

In [23]:
excel_list = pd.read_excel('./Cat_datos_gob_population.xlsx')
excel_list.head(3)

,URL,IDENTIFICADOR,TÃTULO,DESCRIPCIÃ“N,TEMÃTICAS,ETIQUETAS,FECHA DE CREACIÃ“N,FECHA DE ÃšLTIMA MODIFICACIÃ“N,FRECUENCIA DE ACTUALIZACIÃ“N,IDIOMAS,Ã“RGANO PUBLICADOR,CONDICIONES DE USO,COBERTURA GEOGRÃFICA,COBERTURA TEMPORAL,VIGENCIA DEL RECURSO,RECURSOS RELACIONADOS,NORMATIVA,DISTRIBUCIONES
0,https://datos.gob.es/catalogo/617e5d5b-bd3a-43...,urn:ine:es:TABLA:T3:65:33573,"[en]Population by sex, municipalities and coun...","[en]Table of INEBase Population by sex, munici...",DemografÃ­a,[en]Abroad//Africa//Algeria//America//Brazil//...,2012-01-13T12:00:00+0100,2022-01-17T12:00:00+0100,[TYPE]http://www.w3.org/2006/time#years[VALUE]1,es//en,Instituto Nacional de EstadÃ­stica,https://www.ine.es/aviso_legal,EspaÃ±a,2003-01-01T01:00:00+0100-2021-01-01T01:00:00+0100,NaN,https://www.ine.es/dyngs/IOE/es/operacion.htm?...,https://www.ine.es/dynt3/metadatos/es/Respuest...,[TITLE_en]CSV: Tab Separated[TITLE_es]CSV: sep...
1,https://datos.gob.es/catalogo/023beec4-72ec-40...,urn:ine:es:TABLA:T3:65:33579,"[en]Population by sex, municipalities and coun...","[en]Table of INEBase Population by sex, munici...",DemografÃ­a,[en]Abroad//Africa//Algeria//America//Brazil//...,2012-01-13T12:00:00+0100,2022-01-17T12:00:00+0100,[TYPE]http://www.w3.org/2006/time#years[VALUE]1,es//en,Instituto Nacional de EstadÃ­stica,https://www.ine.es/aviso_legal,Albacete,2003-01-01T01:00:00+0100-2021-01-01T01:00:00+0100,NaN,https://www.ine.es/dyngs/IOE/es/operacion.htm?...,https://www.ine.es/dynt3/metadatos/es/Respuest...,[TITLE_en]Plain Text: separated by semicolons[...
2,https://datos.gob.es/catalogo/d7b4b78f-cef5-4d...,urn:ine:es:TABLA:T3:65:33589,"[en]Population by sex, municipalities and coun...","[en]Table of INEBase Population by sex, munici...",DemografÃ­a,[en]Abroad//Africa//Algeria//America//Brazil//...,2012-01-13T12:00:00+0100,2022-01-17T12:00:00+0100,[TYPE]http://www.w3.org/2006/time#years[VALUE]1,es//en,Instituto Nacional de EstadÃ­stica,https://www.ine.es/aviso_legal,Alicante,2003-01-01T01:00:00+0100-2021-01-01T01:00:00+0100,NaN,https://www.ine.es/dyngs/IOE/es/operacion.htm?...,https://www.ine.es/dynt3/metadatos/es/Respuest...,[TITLE_en]Html[TITLE_es]Html[ACCESS_URL]https:...


In [21]:
distribuciones = []
for i in excel_list.index:
    dist = excel_list.iloc[i].to_dict()['DISTRIBUCIONES'].split('[TITLE_en]')
    
    for resource in dist:
        
        if resource[:4] == 'Json':
            link = resource.split('[ACCESS_URL]')[1].split('[MEDIA_TYPE]')[0]
    
            distribuciones.append(link)

In [22]:
distribuciones

['https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33589?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33659?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33695?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33743?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33761?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33767?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33779?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33803?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33809?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33815?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33833?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33869?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33881?tip=AM',
 'https://servicios.ine.es/wstempu

In [25]:
basic_headers = {'User-Agent': 'Mozilla/5.0'}
population_all = []

for link in distribuciones:
    payload = requests.get(link, headers=basic_headers)
    payload_json = payload.json()
    
    for reg in payload_json:

        try:
            if reg['MetaData'][1]['Variable']['Codigo'] == 'MUN':
                temp = {"id": reg['COD']}

                temp['Poblacion'] = reg['MetaData'][1]['Nombre']
                temp['Origen'] = reg['MetaData'][2]['Nombre']

                for row in reg['Data']:
                    temp[datetime.datetime.strptime(row['Fecha'], '%Y-%m-%dT%H:%M:%S.000+01:00').year] = int(row['Valor'])

                population_all.append(temp)
        except Exception as e:
            pass

In [26]:
population_all_std = pd.DataFrame(population_all, columns=population_all[0].keys())
population_all_std.head(20)

,id,Poblacion,Origen,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5663044,Agost,Total,4906.0,4794.0,4758.0,4713.0,4703.0,4729.0,4707.0,...,4791.0,4831.0,4818.0,4810.0,4813.0,4766.0,4723.0,4601.0,4517.0,4451.0
1,PC5663043,Agost,España,4426.0,4329.0,4321.0,4295.0,4298.0,4308.0,4290.0,...,4290.0,4300.0,4279.0,4243.0,4264.0,4260.0,4220.0,4147.0,4111.0,4081.0
2,PC5663042,Agost,Extranjero,480.0,465.0,437.0,418.0,405.0,421.0,417.0,...,501.0,531.0,539.0,567.0,549.0,506.0,503.0,454.0,406.0,370.0
3,PC5663013,Agost,Europa (sin España),196.0,191.0,188.0,187.0,186.0,193.0,194.0,...,251.0,254.0,250.0,257.0,261.0,234.0,207.0,181.0,145.0,132.0
4,PC5663012,Agost,UE28 sin España,NaN,157.0,153.0,151.0,151.0,159.0,162.0,...,218.0,214.0,210.0,212.0,214.0,183.0,145.0,131.0,99.0,96.0
5,PC5663041,Agost,Alemania,20.0,22.0,19.0,19.0,21.0,20.0,21.0,...,21.0,22.0,24.0,23.0,21.0,20.0,14.0,14.0,16.0,16.0
6,PC5663040,Agost,Bulgaria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
7,PC5663039,Agost,Francia,39.0,31.0,28.0,28.0,26.0,29.0,28.0,...,32.0,29.0,29.0,30.0,30.0,30.0,31.0,30.0,26.0,28.0
8,PC5663038,Agost,Italia,7.0,8.0,8.0,7.0,6.0,6.0,4.0,...,3.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,PC5663037,Agost,Polonia,2.0,2.0,2.0,2.0,2.0,3.0,4.0,...,11.0,10.0,9.0,8.0,8.0,8.0,8.0,5.0,4.0,6.0
